In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

pd.set_option('display.max_columns',None)

In [ ]:
df_home=pd.read_csv('/Users/aurelientarroux/Desktop/Projet_prog/Projet_1/Data/Train_Data/train_home_team_statistics_df.csv')
df_away=pd.read_csv('/Users/aurelientarroux/Desktop/Projet_prog/Projet_1/Data/Train_Data/train_away_team_statistics_df.csv')

df_home.columns='home_'+df_home.columns
df_away.columns='away_'+df_away.columns

df_result=pd.read_csv('/Users/aurelientarroux/Desktop/Projet_prog/Projet_1/Data/Y_train_1rknArQ.csv')


In [ ]:
base_match=pd.merge(df_home,df_away,left_on="home_ID",right_on='away_ID',how='inner')
base=pd.merge(base_match,df_result,left_on="home_ID",right_on='ID',how='inner')
base=base.drop(['home_ID','home_LEAGUE','home_TEAM_NAME','away_ID','away_LEAGUE','away_TEAM_NAME'],axis=1)


In [ ]:
def resultat(row):
    if row['HOME_WINS']==1:
        return 'HOME'
    elif row['AWAY_WINS']==1:
        return 'AWAY'
    else:
        return 'DRAW'

base['Resultat']=base.apply(resultat, axis=1)
print(resultat(base.iloc[0]))
base=base.drop(['HOME_WINS','AWAY_WINS','DRAW'],axis=1)



In [ ]:
base.dropna(inplace=True)

In [ ]:
X = base.drop(['Resultat','ID'],axis=1)
y = base['Resultat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Centre et réduit mes données
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#J'entraine mon modèle
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)  # 'ovr' pour One-vs-Rest, le paramètre par défaut
model.fit(X_train, y_train)

#Je prédis
y_pred = model.predict(X_test)

#Je regarde les résultat
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)
print(classification_report(y_test, y_pred)) 
print(confusion_matrix(y_test, y_pred))

In [ ]:
#Je regarde quelle sont les variables qui ont le plus joué 
coefficients=model.coef_
coefficients=pd.DataFrame(coefficients)
coefficients.columns=X.columns

coefficients.applymap(abs)
coefficients=pd.DataFrame(coefficients.sum(axis=0))
coefficients.sort_values(by=0,ascending=False)

In [ ]:
#Proba pour chaque individus d'aprtenir aux classes

proba_list = model.predict_proba(X)
df_proba=pd.DataFrame(proba_list)

sns.histplot(df_proba.max(axis=1))